In [1]:
import sys
from pprint import pprint
sys.path.append('/apps/binary/tfel/4.1.0/lib/python3.11/site-packages')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import std
import tfel.tests
from mtest import *
from tfel.material import ModellingHypothesis
import mfront

In [6]:
import std
import tfel.tests
from mtest import *

m = MTest()

setVerboseMode(VerboseLevel.VERBOSE_LEVEL0)
m.setPredictionPolicy(PredictionPolicy.LINEARPREDICTION)

m.setBehaviour("aster","/apps/binary/tfel/behaviors/src/libAsterBehaviour.so", "asterfullchabochememory")
m.setMaterialProperty('YoungModulus', 201000)
m.setMaterialProperty('PoissonRatio', 0.3)
m.setMaterialProperty('R_0', 180.)
m.setMaterialProperty('C[0]', 90000.)
m.setMaterialProperty('C[1]', 9000.)
m.setMaterialProperty('g[0]', 630.)
m.setMaterialProperty('g[1]', 100.)
m.setMaterialProperty('ainf', 0.5)
m.setMaterialProperty('b', 10.)
m.setMaterialProperty('m', 7.5)
m.setMaterialProperty('K', 33.306)
m.setMaterialProperty('Eta', 0.5)
m.setMaterialProperty('Qm', -100)
m.setMaterialProperty('Q0', 620)
m.setMaterialProperty('Mu', 12)
m.setMaterialProperty('delta[0]', 1)
m.setMaterialProperty('delta[1]', 1)
m.setExternalStateVariable('Temperature', 293.15)

time = np.loadtxt("time.txt")
time = time[:,0].tolist()
strain_up = 0.0208
strain_down = 0.0008

m.setTimes(time)
m.setImposedStrain('EXX', {0.: 0.0, 0.1 :strain_up, 1.0:strain_up,
                           61.:strain_down,121.:strain_up,181.:strain_down,241.:strain_up,
                           301.:strain_down,361.:strain_up,421.:strain_down,481.:strain_up} )

m.setOutputFileName("fullchabochememory.res")
m.execute()
del m

In [ ]:
test = MTest()
setVerboseMode(VerboseLevel.VERBOSE_LEVEL2)
test.setPredictionPolicy(PredictionPolicy.LINEARPREDICTION)
materialParser(test, parser)

strain_up = 0.0208
strain_down = 0.0008

time = np.loadtxt("time.txt")
time = time[:,0].tolist()

calculate(test, parser)

In [ ]:
behavior_law_folder="behavior_laws/"
behavior_law = 'FullChabocheMemory'
general_interface = 'aster'
fronfile = behavior_law_folder + behavior_law + '/' + behavior_law + ".mfront"

dsl = mfront.getDSL(fronfile)
parser = prepare_behaviour(dsl, fronfile, general_interface)
#compile(dsl)
print(fronfile)

In [ ]:
def compile(fronobject):
    fronobject.generateOutputFiles()
    internal = mfront.CMakeGenerator()
    internal.exe(fronobject.getTargetsDescription())

def prepare_behaviour(fronobject, inputFile, solverInterface):
    info = fronobject.getFileDescription()
    fronobject.setInterfaces([solverInterface])
    fronobject.analyseFile(inputFile,[])
    behaviour = str(info.description)
    behaviour = behaviour.replace(" ", "")
    behaviour = behaviour.replace('*', "")
    behaviour = behaviour.upper()
    #desc = fronobject.getBehaviourDescription()
    #info2 = desc.getBehaviourData(ModellingHypothesis.TRIDIMENSIONAL)
    #for mp in info2.getMaterialProperties():
    #    print(info2.getExternalName(mp.name))
    return behaviour

def materialParser(fronobject, behaviourName):
    fronobject.setBehaviour("aster","behavior_laws/FullChabocheMemory/src/libAsterBehaviour.so", "asterfullchabochememory")
    fronobject.setMaterialProperty('YoungModulus', 201000)
    fronobject.setMaterialProperty('PoissonRatio', 0.3)
    fronobject.setMaterialProperty('R_0', 180.)
    fronobject.setMaterialProperty('C[0]', 90000.)
    fronobject.setMaterialProperty('C[1]', 9000.)
    fronobject.setMaterialProperty('g[0]', 630.)
    fronobject.setMaterialProperty('g[1]', 100.)
    fronobject.setMaterialProperty('ainf', 0.5)
    fronobject.setMaterialProperty('b', 10.)
    fronobject.setMaterialProperty('m', 7.5)
    fronobject.setMaterialProperty('K', 33.306)
    fronobject.setMaterialProperty('Eta', 0.5)
    fronobject.setMaterialProperty('Qm', -100)
    fronobject.setMaterialProperty('Q0', 620)
    fronobject.setMaterialProperty('Mu', 12)
    fronobject.setMaterialProperty('delta[0]', 1)
    fronobject.setMaterialProperty('delta[1]', 1)
    fronobject.setExternalStateVariable('Temperature', 293.15)

def calculate(fronobject, behaviourName):
    fronobject.setTimes(time)
    fronobject.setImposedStrain('EXX', {0.: 0.0, 0.1 :strain_up, 1.0:strain_up,
                                        61.:strain_down,121.:strain_up,181.:strain_down,241.:strain_up,
                                        301.:strain_down,361.:strain_up,421.:strain_down,481.:strain_up} )
    fronobject.setOutputFileName(behaviourName+".results")
    fronobject.execute()